# Ejercicio: Crear un DataFrame Completo con SWAPI

## Objetivo:
El objetivo de este ejercicio es que los alumnos practiquen la obtención de datos de una API REST y su posterior procesamiento para crear un DataFrame en Pandas que contenga información detallada sobre personajes de Star Wars.

## Instrucciones:

1. Obtener los datos:

- Utilizar la API de Star Wars (SWAPI) para obtener datos sobre los personajes (people) de la saga.
- La URL base para obtener la lista de personajes es: https://swapi.tech/api/.
- Dado que los datos están paginados, deberán recorrer todas las páginas para obtener la lista completa de personajes.

2. Crear el DataFrame:

    Crear un DataFrame que incluya las siguientes columnas con la información detallada de cada personaje:

    - id: Identificador único del personaje.
    - name: Nombre del personaje.
    - height: Altura del personaje (en centímetros).
    - mass: Peso del personaje (en kilogramos).
    - hair_color: Color de cabello.
    - skin_color: Color de piel.
    - eye_color: Color de ojos.
    - birth_year: Año de nacimiento.
    - gender: Género.
    - homeworld: Nombre del planeta de origen (obtenido mediante una petición adicional a la API).
    - species: Nombre de la especie del personaje (obtenido mediante una petición adicional si el campo species no está vacío).
    - films: Lista de nombres de películas en las que aparece el personaje (obtenido mediante peticiones adicionales).

3. Procesamiento de datos:

    - Cada entrada en el campo homeworld de los personajes es un enlace a la API, desde el cual se debe obtener el nombre del planeta.

    - El campo species contiene una lista de enlaces. Si no está vacío, se debe obtener el nombre de la especie desde la API.

    - El campo films contiene una lista de enlaces a las películas. Se debe realizar una petición por cada enlace para obtener el título de la película.

4. Entrega del ejercicio:

    - Se debe entregar un archivo .ipynb (Jupyter Notebook) con el código que obtenga los datos, procese las respuestas y construya el DataFrame completo.

    - También deben incluir en el notebook una breve descripción de cómo organizaron el proceso para obtener los datos y construir el DataFrame.

Tips:

- Utilicen la librería requests para realizar las peticiones HTTP.
- Tengan en cuenta que la API utiliza paginación, por lo que deberán iterar sobre las páginas hasta obtener todos los datos.
- Los datos relacionados (homeworld, species, films) requieren realizar peticiones adicionales, así que gestionen adecuadamente los tiempos de espera (pueden considerar utilizar la función time.sleep() para evitar sobrecargar la API).
- Si encuentran campos con datos faltantes o errores, documenten cómo lo manejan en su notebook.

In [1]:
import hashlib
import requests
import datetime
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://swapi.dev/api/people/'
response = requests.get("https://swapi.dev/api/people/", verify=False)
print(f"Status Code: {response.status_code}")
print(f"Content: {response.text}")

Status Code: 200
Content: {"count":82,"next":"https://swapi.dev/api/people/?page=2","previous":null,"results":[{"name":"Luke Skywalker","height":"172","mass":"77","hair_color":"blond","skin_color":"fair","eye_color":"blue","birth_year":"19BBY","gender":"male","homeworld":"https://swapi.dev/api/planets/1/","films":["https://swapi.dev/api/films/1/","https://swapi.dev/api/films/2/","https://swapi.dev/api/films/3/","https://swapi.dev/api/films/6/"],"species":[],"vehicles":["https://swapi.dev/api/vehicles/14/","https://swapi.dev/api/vehicles/30/"],"starships":["https://swapi.dev/api/starships/12/","https://swapi.dev/api/starships/22/"],"created":"2014-12-09T13:50:51.644000Z","edited":"2014-12-20T21:17:56.891000Z","url":"https://swapi.dev/api/people/1/"},{"name":"C-3PO","height":"167","mass":"75","hair_color":"n/a","skin_color":"gold","eye_color":"yellow","birth_year":"112BBY","gender":"n/a","homeworld":"https://swapi.dev/api/planets/1/","films":["https://swapi.dev/api/films/1/","https://swa

In [3]:
all_characters = []

while url:
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "results" in data:
            all_characters.extend(data["results"])
        url = data.get("next")
    else:
        print("Error al obtener datos de la API")
        break


In [4]:
def get_species_name(species_url):
    response = requests.get(species_url)
    if response.status_code == 200:
        species_data = response.json()
        return species_data['name']
    return None

In [5]:
def get_film_titles(film_urls):
    film_titles = []
    for film_url in film_urls:
        response = requests.get(film_url)
        if response.status_code == 200:
            film_data = response.json()
            film_titles.append(film_data['title'])
    return film_titles


In [6]:
def get_homeworld_name(homeworld_url):
    response = requests.get(homeworld_url)
    if response.status_code == 200:
        homeworld_data = response.json()
        return homeworld_data['name']
    return None

In [7]:
character_list = []

for character in all_characters:
    species_names = [get_species_name(url) for url in character['species']] if character['species'] else []

    film_titles = get_film_titles(character['films'])

    homeworld_name = get_homeworld_name(character['homeworld'])

    character_data = {
        'name': character['name'],
        'height': character['height'],
        'mass': character['mass'],
        'hair_color': character['hair_color'],
        'skin_color': character['skin_color'],
        'eye_color': character['eye_color'],
        'birth_year': character['birth_year'],
        'gender': character['gender'],
        'homeworld': homeworld_name,
        'species': species_names,
        'films': film_titles
    }
    character_list.append(character_data)

df = pd.DataFrame(character_list)

print(df)

               name height mass hair_color    skin_color      eye_color  \
0    Luke Skywalker    172   77      blond          fair           blue   
1             C-3PO    167   75        n/a          gold         yellow   
2             R2-D2     96   32        n/a   white, blue            red   
3       Darth Vader    202  136       none         white         yellow   
4       Leia Organa    150   49      brown         light          brown   
..              ...    ...  ...        ...           ...            ...   
77         Grievous    216  159       none  brown, white  green, yellow   
78          Tarfful    234  136      brown         brown           blue   
79  Raymus Antilles    188   79      brown         light          brown   
80        Sly Moore    178   48       none          pale          white   
81       Tion Medon    206   80       none          grey          black   

   birth_year  gender homeworld    species  \
0       19BBY    male  Tatooine         []   
1      

In [10]:
df.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species,films
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,Tatooine,[],"[A New Hope, The Empire Strikes Back, Return o..."
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,Tatooine,[Droid],"[A New Hope, The Empire Strikes Back, Return o..."
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,Naboo,[Droid],"[A New Hope, The Empire Strikes Back, Return o..."
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,Tatooine,[],"[A New Hope, The Empire Strikes Back, Return o..."
4,Leia Organa,150,49,brown,light,brown,19BBY,female,Alderaan,[],"[A New Hope, The Empire Strikes Back, Return o..."
